In [5]:
import numpy as np
import networkx as nx
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import json
from mesh_utils import load_and_process_mesh
import matplotlib.pyplot as plt
import igl
import scipy.sparse as sp

### Load Meshes

In [18]:
base_dir = Path("/Users/schimmenti/Desktop/DresdenProjects/wingsurface/final_meshes/wildtype/")
json_file_with_infos = base_dir.joinpath("mesh_infos.json")
mesh_infos_dict = json.load(open(json_file_with_infos))
mesh_infos_dict = { identifier : info for identifier, info in sorted(mesh_infos_dict.items())}

In [31]:
def load_mesh(mesh_info):
    scale = mesh_info['scale'] if 'scale' in mesh_info else 1.0
    flip0 = mesh_info['flip0'] if 'flip0' in mesh_info else False
    flip1 = mesh_info['flip1'] if 'flip1' in mesh_info else False
    flip2 = mesh_info['flip2'] if 'flip2' in mesh_info else False
    vertices, triangles  = load_and_process_mesh(base_dir.joinpath(mesh_info['filename']), scale=scale, flip0=flip0, flip1=flip1, flip2=flip2, return_as_open3d_mesh=False)
    #if(loaded_mesh.is_vertex_manifold()==False):
    #    print(f"Mesh {mesh_identifier} is not vertex manifold!")
    #if(loaded_mesh.is_edge_manifold()==False):
    #    print(f"Mesh {mesh_identifier} is not edge manifold!")
    #vertices, triangles = np.asarray(loaded_mesh.vertices), np.asarray(loaded_mesh.triangles)
    adjList = igl.adjacency_list(triangles)
    adjGraph = nx.from_dict_of_lists({ i : list(adjList[i])  for i in range(len(adjList))})
    cotmatrix = igl.cotmatrix(vertices, triangles)
    massmatrix = igl.massmatrix(vertices, triangles, igl.MASSMATRIX_TYPE_VORONOI)
    vertex_normals = igl.per_vertex_normals(vertices, triangles)
    laplacian = sp.linalg.inv(massmatrix) @ cotmatrix
    mean_curvature = np.sum((laplacian @ vertices)*vertex_normals, axis=1)
    pc_res = igl.principal_curvature(vertices, triangles)[:4]
    mesh_dict = {}
    mesh_dict['vertices'] = vertices
    mesh_dict['triangles'] = triangles
    #mesh_dict['adjList'] = adjList
    mesh_dict['adjGraph'] = adjGraph
    mesh_dict['cotmatrix'] = cotmatrix
    mesh_dict['massmatrix'] = massmatrix
    mesh_dict['laplacian'] = laplacian
    mesh_dict['vertex_normals'] = vertex_normals
    mesh_dict['mean_curvature'] = mean_curvature
    mesh_dict['principal_curvature_1'] = pc_res[0]
    mesh_dict['principal_curvature_2'] = pc_res[1]
    return mesh_dict

In [32]:
meshes_dict = {}
for mesh_identifier, mesh_info in mesh_infos_dict.items():
    scale = mesh_info['scale'] if 'scale' in mesh_info else 1.0
    flip0 = mesh_info['flip0'] if 'flip0' in mesh_info else False
    flip1 = mesh_info['flip1'] if 'flip1' in mesh_info else False
    flip2 = mesh_info['flip2'] if 'flip2' in mesh_info else False
    mesh_dict = load_mesh(mesh_info)
    break
    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.plot_trisurf(vertices[:, 1], vertices[:, 2], triangles, vertices[:, 0], cmap=plt.cm.Spectral, linewidth=0.2)
    #ax.set_title(mesh_identifier)
    #ax.set_aspect('equal')
    #plt.show()

{'vertices': array([[ 14.79540388, 209.38479148, 124.51842279],
        [ 16.55548715, 209.79033882, 126.67640735],
        [ 15.92880272, 208.60573206, 123.89388882],
        ...,
        [194.07180059, 124.01869516, 109.20084845],
        [194.13260618, 123.9766419 , 111.19012471],
        [193.99635868, 125.05974324, 114.93139169]], shape=(25472, 3)),
 'triangles': array([[22039, 21777, 21858],
        [25470, 25462, 25471],
        [25470, 25467, 25462],
        ...,
        [   10,    12,    13],
        [    2,     5,     1],
        [    2,     1,     0]], shape=(50518, 3), dtype=int32),
 'adjGraph': <networkx.classes.graph.Graph at 0x17aae4320>,
 'cotmatrix': <Compressed Sparse Column sparse matrix of dtype 'float64'
 	with 177450 stored elements and shape (25472, 25472)>,
 'massmatrix': <Compressed Sparse Column sparse matrix of dtype 'float64'
 	with 25472 stored elements and shape (25472, 25472)>,
 'laplacian': <Compressed Sparse Column sparse matrix of dtype 'float64'
 	wit